<a href="https://www.kaggle.com/code/aleksandr13254/mysolve?scriptVersionId=109368167" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm # loops show a smart progress meter
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
import copy
import plotly.graph_objects as go
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-cpi-test/CPI_MoM.csv
/kaggle/input/dataset-cpi-test/CPI_YoY.csv
/kaggle/input/dataset-cpi-test/CPI_Russia.csv
/kaggle/input/dataset-cpi-test/Product_Price_Month.csv
/kaggle/input/dataset-cpi-test/Product_Price_Day.csv
/kaggle/input/dataset-cpi-test/Product_CPI.csv
/kaggle/input/dataset/DS_tests/DS_test9(2020-06--2022-09-01).csv
/kaggle/input/dataset/DS_tests/DS_test8(2020-06--2022-08-21).csv
/kaggle/input/dataset/DS_tests/DS_test5(2020-06--2022-07-21).csv
/kaggle/input/dataset/DS_tests/DS_test4(2020-06--2022-07-11).csv
/kaggle/input/dataset/DS_tests/DS_test7(2020-06--2022-08-11).csv
/kaggle/input/dataset/DS_tests/DS_test1(2020-06--2022-06-11).csv
/kaggle/input/dataset/DS_tests/DS_test3(2020-06--2022-07-01).csv
/kaggle/input/dataset/DS_tests/DS_test6(2020-06--2022-08-01).csv
/kaggle/input/dataset/DS_tests/DS_test2(2020-06--2022-06-21).csv
/kaggle/input/dataset/DS_train/DS_train(2020-06--2022-06-01).csv


In [47]:
df = pd.read_csv('../input/dataset/DS_tests/DS_test9(2020-06--2022-09-01).csv', delimiter='\t',parse_dates=['DateObserve'])
# df = pd.read_csv('../input/dataset/DS_train/DS_train(2020-06--2022-06-01).csv', delimiter='\t',parse_dates=['DateObserve'], nrows=107892)
df

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df = df[df['StockStatus']=='InStock'][['WebPriceId','DateObserve','CurrentPrice']]
df

In [ ]:
df.dropna()
df

In [ ]:
df['DateObserve'] = pd.to_datetime(df['DateObserve']).apply(lambda x: x.strftime('%Y-%m-%d'))
df

In [ ]:
df = df.groupby(['WebPriceId','DateObserve']).agg({'CurrentPrice': ['mean']}).reset_index()
df.columns = ['Id','Date','Price']
df

In [ ]:
df = df[df['Price'] < 100000]
df

In [ ]:
df = df[df['Price'] > 10]
df

In [ ]:
count_filtered = df.groupby(['Id'])['Id'].count()
count_filtered

In [ ]:
filtered_ids = pd.DataFrame(count_filtered[count_filtered > 60].index)
filtered_ids

In [ ]:
df = filtered_ids.merge(df, on='Id',how='inner')
df

In [ ]:
df.to_csv('Product_Price_Day.csv',sep=';',encoding='utf-8')
# df = pd.read_csv('../input/dataset-cpi-test/Product_Price_Day.csv', delimiter=';', index_col=[0])
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date']).apply(lambda x: x.strftime('%Y-%m'))
df

In [ ]:
df = df.groupby(['Id','Date']).agg({'Price': ['mean']}).reset_index()
df.columns = ['Id','Date','Price']
df

In [ ]:
df.to_csv('Product_Price_Month.csv',sep=';',encoding='utf-8')
# df = pd.read_csv('../input/dataset-cpi-test/Product_Price_Month.csv', delimiter=';', index_col=[0])
df

In [ ]:
date_list_all = df.sort_values(by=['Date']).Date.unique()
date_list_all

In [ ]:
id_product_list = df['Id'].unique()[:-1]
len(id_product_list)

In [ ]:
id_list = list()
date_start_list = list()
date_end_list = list()
cpi_product_list = list()
for id_product in tqdm(id_product_list):
    df_id_product = df.loc[df.Id == id_product]
    date_list = df_id_product.sort_values(by=['Date']).Date.unique()
    for i in range(0, len(date_list)):        
        start_value = df_id_product.loc[df_id_product.Date == date_list[i]].Price.values[0]
        for j in range(i+1, len(date_list)):
            end_value = df_id_product.loc[df_id_product.Date == date_list[j]].Price.values[0]
            cpi_product = end_value/start_value*100
            id_list.append(id_product)
            date_start_list.append(date_list[i])
            date_end_list.append(date_list[j])
            cpi_product_list.append(cpi_product)

In [ ]:
cpi_id_product = pd.DataFrame({'Id': id_list, 'Date_start': date_start_list, 'Date_end': date_end_list, 'CPI': cpi_product_list})
cpi_id_product

In [ ]:
cpi_id_product.to_csv('Product_CPI.csv',sep=';',encoding='utf-8')
# cpi_id_product = pd.read_csv('../input/dataset-cpi-test/Product_CPI.csv', delimiter=';', index_col=[0])
cpi_id_product

In [ ]:
cpi_id_product = cpi_id_product[cpi_id_product.CPI < 120][cpi_id_product.CPI > 80]
cpi_id_product

In [ ]:
cpi_count = cpi_id_product[['Id','Date_start','Date_end',]].groupby(['Date_end','Date_start']).count().reset_index()
cpi_count.columns = ['Date_end','Date_start','Count_Id',]
cpi_count

In [ ]:
date_list_year = ['2020', '2021', '2022']
date_list_month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
date_list_year_month = list()
cpi_list = list()
cpi_date_list = list()
for y in date_list_year:
    for m in date_list_month:
        date_list_year_month.append(f'{y}-{m}')
for i in range(1, len(date_list_year_month)):
    item = cpi_count.loc[(cpi_count.Date_start == date_list_year_month[i-1]) & (cpi_count.Date_end == date_list_year_month[i])]
    if item.empty == False:
        cpi_list.append(item.Count_Id.values[0])
        cpi_date_list.append(item.Date_end.values[0])
cpi_id_count = pd.DataFrame({'Date': cpi_date_list, 'Count_Id': cpi_list})
cpi_id_count

In [ ]:
cpi_df = cpi_id_product.groupby(['Date_start','Date_end']).agg({'CPI': ['mean']}).reset_index()
cpi_df.columns = ['Date_start','Date_end','CPI']
cpi_df

In [ ]:
date_list_year = ['2020', '2021', '2022']
date_list_month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
date_list_year_month = list()
cpi_list = list()
cpi_date_list = list()
for y in date_list_year:
    for m in date_list_month:
        date_list_year_month.append(f'{y}-{m}')
for i in range(1, len(date_list_year_month)):
    item = cpi_df.loc[(cpi_df.Date_start == date_list_year_month[i-1]) & (cpi_df.Date_end == date_list_year_month[i])]
    if item.empty == False:
        cpi_list.append(item.CPI.values[0])
        cpi_date_list.append(item.Date_end.values[0])
cpi_mom = pd.DataFrame({'Date': cpi_date_list, 'MoM': cpi_list})
cpi_mom

In [ ]:
cpi_yoy = cpi_mom.copy()
for i in range(12, len(cpi_yoy)):
  temp = 100
  for j in range(11, -1, -1):
    temp *= float(cpi_mom.iloc[i-j, 1])/100
  cpi_yoy.iloc[i, 1] = temp
cpi_yoy = cpi_yoy.iloc[12:, :].reset_index(drop = True)
cpi_yoy.columns = ['Date','YoY']
cpi_yoy

In [2]:
# cpi_mom.to_csv('CPI_MoM.csv',sep=';',encoding='utf-8')
cpi_mom = pd.read_csv('../input/dataset-cpi-test/CPI_MoM.csv', delimiter=';', index_col=[0])
cpi_mom

,Date,MoM
0,2020-07,99.851962
1,2020-08,99.337024
2,2020-09,100.570486
3,2020-10,99.910327
4,2020-11,97.395858
5,2020-12,101.445817
6,2021-01,101.144785
7,2021-02,99.449539
8,2021-03,99.582982
9,2021-04,99.711704


In [3]:
# cpi_yoy.to_csv('CPI_YoY.csv',sep=';',encoding='utf-8')
cpi_yoy = pd.read_csv('../input/dataset-cpi-test/CPI_YoY.csv', delimiter=';', index_col=[0])
cpi_yoy = cpi_yoy.iloc[:,1:].reset_index(drop = True)
cpi_yoy

,YoY
0,99.079655
1,99.651411
2,100.461306
3,104.488735
4,110.284777
5,109.804602
6,110.881921
7,111.630367
8,119.660790
9,116.932430


In [4]:
cpi_russia = pd.read_csv('../input/dataset-cpi-test/CPI_Russia.csv', delimiter=';')
cpi_russia = cpi_russia
cpi_russia

,Date,Russia_MoM,Russia_YoY,CFD_MoM,CFD_YoY,Iv_MoM,Iv_YoY
0,2020-07,100.35,103.37,100.36,103.25,100.48,103.92
1,2020-08,99.96,103.58,99.87,103.42,100.02,104.48
2,2020-09,99.93,103.67,99.89,103.48,99.86,104.44
3,2020-10,100.43,103.99,100.38,103.78,100.56,105.06
4,2020-11,100.71,104.42,100.65,104.16,100.73,105.43
5,2020-12,100.83,104.91,100.84,104.70,100.82,105.79
6,2021-01,100.67,105.19,100.79,105.11,101.00,106.34
7,2021-02,100.78,105.67,100.71,105.55,101.03,106.99
8,2021-03,100.66,105.79,100.63,105.71,100.58,106.64
9,2021-04,100.58,105.53,100.54,105.54,100.78,106.53


In [5]:
df_14 = pd.concat([cpi_russia.iloc[12:,1:].reset_index(drop = True), cpi_mom.iloc[12:,1:].reset_index(drop = True), cpi_yoy], axis=1)
df_14

,Russia_MoM,Russia_YoY,CFD_MoM,CFD_YoY,Iv_MoM,Iv_YoY,MoM,YoY
0,100.31,106.46,100.26,106.30,100.31,107.57,100.771324,99.079655
1,100.17,106.68,100.10,106.53,100.29,107.87,99.910265,99.651411
2,100.60,107.40,100.66,107.35,100.53,108.60,101.387850,100.461306
3,101.11,108.13,101.02,108.03,101.51,109.63,103.915668,104.488735
4,100.96,108.40,100.96,108.36,100.86,109.77,102.798455,110.284777
5,100.82,108.39,100.80,108.32,100.97,109.94,101.004126,109.804602
6,100.99,108.73,100.96,108.50,101.22,110.18,102.137140,110.881921
7,101.17,109.15,101.42,109.26,101.13,110.28,100.120816,111.630367
8,107.61,116.69,107.37,116.58,109.02,119.54,106.746746,119.660790
9,101.56,117.83,101.74,117.97,102.40,121.46,97.438199,116.932430


In [6]:
df_13_mom = df_14.iloc[:,[0,-2,-1]].copy()
df_13_mom['Russia_MoM-1'] = df_14['Russia_MoM'].shift(1)
df_13_mom['Russia_YoY-1'] = df_14['Russia_YoY'].shift(1)
df_13_mom = df_13_mom.iloc[1:,:].reset_index(drop = True)
df_13_mom

,Russia_MoM,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,100.17,99.910265,99.651411,100.31,106.46
1,100.60,101.387850,100.461306,100.17,106.68
2,101.11,103.915668,104.488735,100.60,107.40
3,100.96,102.798455,110.284777,101.11,108.13
4,100.82,101.004126,109.804602,100.96,108.40
5,100.99,102.137140,110.881921,100.82,108.39
6,101.17,100.120816,111.630367,100.99,108.73
7,107.61,106.746746,119.660790,101.17,109.15
8,101.56,97.438199,116.932430,107.61,116.69
9,100.12,97.832254,113.383708,101.56,117.83


In [7]:
df_13_yoy = df_14.iloc[:,[1,-2,-1]].copy()
df_13_yoy['Russia_MoM-1'] = df_14['Russia_MoM'].shift(1)
df_13_yoy['Russia_YoY-1'] = df_14['Russia_YoY'].shift(1)
df_13_yoy = df_13_yoy.iloc[1:,:].reset_index(drop = True)
df_13_yoy

,Russia_YoY,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,106.68,99.910265,99.651411,100.31,106.46
1,107.40,101.387850,100.461306,100.17,106.68
2,108.13,103.915668,104.488735,100.60,107.40
3,108.40,102.798455,110.284777,101.11,108.13
4,108.39,101.004126,109.804602,100.96,108.40
5,108.73,102.137140,110.881921,100.82,108.39
6,109.15,100.120816,111.630367,100.99,108.73
7,116.69,106.746746,119.660790,101.17,109.15
8,117.83,97.438199,116.932430,107.61,116.69
9,117.10,97.832254,113.383708,101.56,117.83


In [8]:
df_26 = pd.concat([cpi_russia.iloc[:,1:], cpi_mom.iloc[:,1:]], axis=1)
df_26

,Russia_MoM,Russia_YoY,CFD_MoM,CFD_YoY,Iv_MoM,Iv_YoY,MoM
0,100.35,103.37,100.36,103.25,100.48,103.92,99.851962
1,99.96,103.58,99.87,103.42,100.02,104.48,99.337024
2,99.93,103.67,99.89,103.48,99.86,104.44,100.570486
3,100.43,103.99,100.38,103.78,100.56,105.06,99.910327
4,100.71,104.42,100.65,104.16,100.73,105.43,97.395858
5,100.83,104.91,100.84,104.70,100.82,105.79,101.445817
6,100.67,105.19,100.79,105.11,101.00,106.34,101.144785
7,100.78,105.67,100.71,105.55,101.03,106.99,99.449539
8,100.66,105.79,100.63,105.71,100.58,106.64,99.582982
9,100.58,105.53,100.54,105.54,100.78,106.53,99.711704


In [9]:
df_25 = df_26.iloc[:,[0,-1]].copy()
df_25['Russia_MoM-1'] = df_26['Russia_MoM'].shift(1)
df_25 = df_25.iloc[1:,:].reset_index(drop = True)
df_25

,Russia_MoM,MoM,Russia_MoM-1
0,99.96,99.337024,100.35
1,99.93,100.570486,99.96
2,100.43,99.910327,99.93
3,100.71,97.395858,100.43
4,100.83,101.445817,100.71
5,100.67,101.144785,100.83
6,100.78,99.449539,100.67
7,100.66,99.582982,100.78
8,100.58,99.711704,100.66
9,100.74,100.894241,100.58


In [10]:
scaler = preprocessing.StandardScaler()
names = df_25.columns
df_25_norm = pd.DataFrame(scaler.fit_transform(df_25), columns=names)
df_25_norm

,Russia_MoM,MoM,Russia_MoM-1
0,-0.572304,-0.527321,-0.336815
1,-0.592608,0.087555,-0.604673
2,-0.254207,-0.241532,-0.625278
3,-0.064702,-1.494985,-0.281869
4,0.016514,0.523904,-0.089561
5,-0.091774,0.373840,-0.007143
6,-0.017326,-0.471233,-0.117034
7,-0.098542,-0.404712,-0.041484
8,-0.152687,-0.340545,-0.123902
9,-0.044398,0.248945,-0.178847


In [11]:
df_25_norm_train = df_25_norm.iloc[:-3]
df_25_norm_test = df_25_norm.iloc[-3:]
gb_25 = GradientBoostingRegressor(n_estimators = 200, max_depth = 3)
gb_25.fit(df_25_norm_train.iloc[:, 1:], df_25_norm_train.iloc[:, 0])
df_25_gb_predict = gb_25.predict(df_25_norm.iloc[:, 1:])
df_25_gb_predict

array([-0.57165984, -0.59216484, -0.25425537, -0.06493963,  0.01671039,
       -0.09092903, -0.01830373, -0.0978253 , -0.15202693, -0.04509012,
       -0.07874059, -0.3349132 , -0.43046644, -0.13929548,  0.20611315,
        0.10489401,  0.00947239,  0.12396665,  0.24596516,  4.6048977 ,
        0.51050351, -0.46344519, -0.52266169, -0.40395922, -0.32774901])

In [12]:
gb_25.feature_importances_

array([0.95070828, 0.04929172])

In [13]:
df_25_norm_gb = pd.DataFrame({'Russia_MoM_predict': df_25_gb_predict, 'MoM': df_25_norm.iloc[:, 1], 'Russia_MoM-1': df_25_norm.iloc[:, 2]})
df_25_norm_gb

,Russia_MoM_predict,MoM,Russia_MoM-1
0,-0.571660,-0.527321,-0.336815
1,-0.592165,0.087555,-0.604673
2,-0.254255,-0.241532,-0.625278
3,-0.064940,-1.494985,-0.281869
4,0.016710,0.523904,-0.089561
5,-0.090929,0.373840,-0.007143
6,-0.018304,-0.471233,-0.117034
7,-0.097825,-0.404712,-0.041484
8,-0.152027,-0.340545,-0.123902
9,-0.045090,0.248945,-0.178847


In [14]:
df_25_gb = pd.DataFrame(scaler.inverse_transform(df_25_norm_gb), columns=df_25_norm_gb.columns)
df_25_gb

,Russia_MoM_predict,MoM,Russia_MoM-1
0,99.960952,99.337024,100.35
1,99.930655,100.570486,99.96
2,100.429929,99.910327,99.93
3,100.709649,97.395858,100.43
4,100.830290,101.445817,100.71
5,100.671249,101.144785,100.83
6,100.778556,99.449539,100.67
7,100.661060,99.582982,100.78
8,100.580975,99.711704,100.66
9,100.738978,100.894241,100.58


In [15]:
df_25_norm_train = df_25_norm.iloc[:-3]
df_25_norm_test = df_25_norm.iloc[-3:]
rf_25 = RandomForestRegressor(n_estimators = 100, max_depth = 9)
rf_25.fit(df_25_norm_train.iloc[:, 1:], df_25_norm_train.iloc[:, 0])
df_25_rf_predict = rf_25.predict(df_25_norm.iloc[:, 1:])
df_25_rf_predict

array([-0.45481125, -0.47552141, -0.35281712, -0.16730557, -0.04087887,
       -0.07113645, -0.09996372, -0.10808986, -0.13833842, -0.06998137,
       -0.11598138, -0.24655914, -0.41156358, -0.19160278,  0.13035215,
        0.30334286,  0.02287592,  0.08886416,  0.06213047,  2.72934128,
        0.56411485,  0.12304269, -0.39017662, -0.38286715, -0.3511928 ])

In [16]:
rf_25.feature_importances_

array([0.62144417, 0.37855583])

In [17]:
df_25_norm_rf = pd.DataFrame({'Russia_MoM_predict': df_25_rf_predict, 'MoM': df_25_norm.iloc[:, 1], 'Russia_MoM-1': df_25_norm.iloc[:, 2]})
df_25_norm_rf

,Russia_MoM_predict,MoM,Russia_MoM-1
0,-0.454811,-0.527321,-0.336815
1,-0.475521,0.087555,-0.604673
2,-0.352817,-0.241532,-0.625278
3,-0.167306,-1.494985,-0.281869
4,-0.040879,0.523904,-0.089561
5,-0.071136,0.373840,-0.007143
6,-0.099964,-0.471233,-0.117034
7,-0.108090,-0.404712,-0.041484
8,-0.138338,-0.340545,-0.123902
9,-0.069981,0.248945,-0.178847


In [18]:
df_25_rf = pd.DataFrame(scaler.inverse_transform(df_25_norm_rf), columns=df_25_norm_rf.columns)
df_25_rf

,Russia_MoM_predict,MoM,Russia_MoM-1
0,100.133600,99.337024,100.35
1,100.103000,100.570486,99.96
2,100.284300,99.910327,99.93
3,100.558400,97.395858,100.43
4,100.745200,101.445817,100.71
5,100.700493,101.144785,100.83
6,100.657900,99.449539,100.67
7,100.645893,99.582982,100.78
8,100.601200,99.711704,100.66
9,100.702200,100.894241,100.58


In [19]:
scaler = preprocessing.StandardScaler()
names = df_13_mom.columns
df_13_norm = pd.DataFrame(scaler.fit_transform(df_13_mom), columns=names)
df_13_norm

,Russia_MoM,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,-0.448931,-0.365105,-1.907280,-0.418912,-1.111940
1,-0.233336,0.224650,-1.760327,-0.490500,-1.060631
2,0.022369,1.233588,-1.029559,-0.270621,-0.892710
3,-0.052838,0.787670,0.022118,-0.009834,-0.722456
4,-0.123032,0.071492,-0.065008,-0.086536,-0.659486
5,-0.037797,0.523717,0.130469,-0.158124,-0.661818
6,0.052452,-0.281067,0.266272,-0.071195,-0.582522
7,3.281359,2.363568,1.723373,0.020847,-0.484568
8,0.247992,-1.351791,1.228319,3.313926,1.273941
9,-0.474000,-1.194510,0.584412,0.220273,1.539817


In [20]:
df_13_norm_train = df_13_norm.iloc[:-3]
df_13_norm_test = df_13_norm.iloc[-3:]
gb_13 = GradientBoostingRegressor(n_estimators = 200, max_depth = 3)
gb_13.fit(df_13_norm_train.iloc[:, 1:], df_13_norm_train.iloc[:, 0])
df_13_gb_predict = gb_13.predict(df_13_norm.iloc[:, 1:])
df_13_gb_predict

array([-0.44893065, -0.2333359 ,  0.02236934, -0.05283807, -0.1230316 ,
       -0.03779657,  0.05245231,  3.28135894,  0.24799176, -0.47399973,
       -0.17841036, -0.22387252, -0.22387252])

In [21]:
gb_13.feature_importances_

array([0.63950912, 0.34549166, 0.0035259 , 0.01147332])

In [22]:
df_13_norm_gb = pd.DataFrame({'Russia_MoM_predict': df_13_gb_predict, 'MoM': df_13_norm.iloc[:, 1], 'YoY': df_13_norm.iloc[:, 2], 'Russia_MoM-1': df_13_norm.iloc[:, 3], 'Russia_YoY-1': df_13_norm.iloc[:, 4]})
df_13_norm_gb

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,-0.448931,-0.365105,-1.907280,-0.418912,-1.111940
1,-0.233336,0.224650,-1.760327,-0.490500,-1.060631
2,0.022369,1.233588,-1.029559,-0.270621,-0.892710
3,-0.052838,0.787670,0.022118,-0.009834,-0.722456
4,-0.123032,0.071492,-0.065008,-0.086536,-0.659486
5,-0.037797,0.523717,0.130469,-0.158124,-0.661818
6,0.052452,-0.281067,0.266272,-0.071195,-0.582522
7,3.281359,2.363568,1.723373,0.020847,-0.484568
8,0.247992,-1.351791,1.228319,3.313926,1.273941
9,-0.474000,-1.194510,0.584412,0.220273,1.539817


In [23]:
df_13_gb = pd.DataFrame(scaler.inverse_transform(df_13_norm_gb), columns=df_13_norm_gb.columns)
df_13_gb

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,100.170000,99.910265,99.651411,100.31,106.46
1,100.600000,101.387850,100.461306,100.17,106.68
2,101.110000,103.915668,104.488735,100.60,107.40
3,100.960000,102.798455,110.284777,101.11,108.13
4,100.820000,101.004126,109.804602,100.96,108.40
5,100.990000,102.137140,110.881921,100.82,108.39
6,101.170000,100.120816,111.630367,100.99,108.73
7,107.610000,106.746746,119.660790,101.17,109.15
8,101.560000,97.438199,116.932430,107.61,116.69
9,100.120000,97.832254,113.383708,101.56,117.83


In [24]:
df_13_norm_train = df_13_norm.iloc[:-3]
df_13_norm_test = df_13_norm.iloc[-3:]
rf_13 = RandomForestRegressor(n_estimators = 100, max_depth = 9)
rf_13.fit(df_13_norm_train.iloc[:, 1:], df_13_norm_train.iloc[:, 0])
df_13_rf_predict = rf_13.predict(df_13_norm.iloc[:, 1:])
df_13_rf_predict

array([-0.34830306, -0.20375434, -0.04702201, -0.04797464, -0.08633044,
       -0.0522364 , -0.07194075,  2.00197993,  0.50364695, -0.16098637,
       -0.08372325, -0.07926094, -0.08031384])

In [25]:
rf_13.feature_importances_

array([0.44329035, 0.30506368, 0.08749509, 0.16415088])

In [26]:
df_13_norm_rf = pd.DataFrame({'Russia_MoM_predict': df_13_rf_predict, 'MoM': df_13_norm.iloc[:, 1], 'YoY': df_13_norm.iloc[:, 2], 'Russia_MoM-1': df_13_norm.iloc[:, 3], 'Russia_YoY-1': df_13_norm.iloc[:, 4]})
df_13_norm_rf

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,-0.348303,-0.365105,-1.907280,-0.418912,-1.111940
1,-0.203754,0.224650,-1.760327,-0.490500,-1.060631
2,-0.047022,1.233588,-1.029559,-0.270621,-0.892710
3,-0.047975,0.787670,0.022118,-0.009834,-0.722456
4,-0.086330,0.071492,-0.065008,-0.086536,-0.659486
5,-0.052236,0.523717,0.130469,-0.158124,-0.661818
6,-0.071941,-0.281067,0.266272,-0.071195,-0.582522
7,2.001980,2.363568,1.723373,0.020847,-0.484568
8,0.503647,-1.351791,1.228319,3.313926,1.273941
9,-0.160986,-1.194510,0.584412,0.220273,1.539817


In [27]:
df_13_rf = pd.DataFrame(scaler.inverse_transform(df_13_norm_rf), columns=df_13_norm_rf.columns)
df_13_rf

,Russia_MoM_predict,MoM,YoY,Russia_MoM-1,Russia_YoY-1
0,100.3707,99.910265,99.651411,100.31,106.46
1,100.6590,101.387850,100.461306,100.17,106.68
2,100.9716,103.915668,104.488735,100.60,107.40
3,100.9697,102.798455,110.284777,101.11,108.13
4,100.8932,101.004126,109.804602,100.96,108.40
5,100.9612,102.137140,110.881921,100.82,108.39
6,100.9219,100.120816,111.630367,100.99,108.73
7,105.0583,106.746746,119.660790,101.17,109.15
8,102.0699,97.438199,116.932430,107.61,116.69
9,100.7443,97.832254,113.383708,101.56,117.83


In [28]:
fig1 = go.Figure()
fig1.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[:, 0], y=df_25_gb.iloc[:, 0], line_color="blue", name='Градиентный бустинг',)
)
fig1.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[:, 0], y=df_25_rf.iloc[:, 0], line_color="green", name='Случайный лес')
)
fig1.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[:, 0], y=df_25.iloc[:, 0], line_color="red", name='Фактические данные')
)
fig1.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>')

In [29]:
fig2 = go.Figure()
fig2.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[-len(df_13_gb):, 0], y=df_13_gb.iloc[:, 0], line_color="blue", name='Градиентный бустинг')
)
fig2.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[-len(df_13_gb):, 0], y=df_13_rf.iloc[:, 0], line_color="green", name='Случайный лес')
)
fig2.add_trace(
      go.Scatter(mode='lines+markers', x=cpi_russia.iloc[-len(df_13_gb):, 0], y=df_13_mom.iloc[:, 0], line_color="red", name='Фактические данные')
)
fig2.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>')

In [30]:
mean_absolute_error(df_25.iloc[-3:, 0], df_25_gb.iloc[-3:, 0])

0.6078077625836755

In [31]:
mean_absolute_error(df_25.iloc[-3:, 0], df_25_rf.iloc[-3:, 0])

0.6719000000000032

In [32]:
mean_absolute_error(df_13_mom.iloc[-3:, 0], df_13_gb.iloc[-3:, 0])

1.0690991362895421

In [33]:
mean_absolute_error(df_13_mom.iloc[-3:, 0], df_13_rf.iloc[-3:, 0])

1.3236333333333334

In [34]:
fig3 = px.scatter(x=df_25.iloc[:, 0], y=df_25.iloc[:, 1])
fig3.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>', 
                   xaxis_title="Инфляция товаров из онлайн магазинов", 
                   yaxis_title="Инфляция на товары и услуги в России")
fig3.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

In [35]:
fig3 = px.scatter(x=df_25.iloc[:, 0], y=df_25.iloc[:, 2])
fig3.update_layout(title_text='<b>Прогнозы инфляция, месяц к месяцу в %<b>', 
                   xaxis_title="Инфляция на товары и услуги в России за прошлый месяц", 
                   yaxis_title="Инфляция на товары и услуги в России")
fig3.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

In [120]:
df_25_nn = df_25
df_25_nn['Spread'] = abs(df_25['MoM'] - df_25['Russia_MoM-1'])
df_25_nn

,Russia_MoM,MoM,Russia_MoM-1,Spread
0,99.96,99.337024,100.35,1.012976
1,99.93,100.570486,99.96,0.610486
2,100.43,99.910327,99.93,0.019673
3,100.71,97.395858,100.43,3.034142
4,100.83,101.445817,100.71,0.735817
5,100.67,101.144785,100.83,0.314785
6,100.78,99.449539,100.67,1.220461
7,100.66,99.582982,100.78,1.197018
8,100.58,99.711704,100.66,0.948296
9,100.74,100.894241,100.58,0.314241


In [222]:
scaler_nn = preprocessing.StandardScaler()
names = df_25_nn.columns
df_25_nn_norm = pd.DataFrame(scaler_nn.fit_transform(df_25_nn), columns=names)
df_25_nn_norm

,Russia_MoM,MoM,Russia_MoM-1,Spread
0,-0.572304,-0.527321,-0.336815,-0.303152
1,-0.592608,0.087555,-0.604673,-0.488467
2,-0.254207,-0.241532,-0.625278,-0.760491
3,-0.064702,-1.494985,-0.281869,0.627438
4,0.016514,0.523904,-0.089561,-0.430762
5,-0.091774,0.373840,-0.007143,-0.624614
6,-0.017326,-0.471233,-0.117034,-0.207621
7,-0.098542,-0.404712,-0.041484,-0.218415
8,-0.152687,-0.340545,-0.123902,-0.332932
9,-0.044398,0.248945,-0.178847,-0.624865


In [173]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.l1 = nn.Linear(3, 2)
        self.f = nn.Tanh()
        self.l2 = nn.Linear(2, 1)
        
    def forward(self, x):
        #print(type(x))
        out = self.l1(x)
        #print(f"L1 {out}")
        #out = self.f(out)
        #print(f"f {out}")
        out = self.l2(out)
        #print(f"l2 {out}")
        return out

In [174]:
model = TheModelClass().to(torch.float64)
print(model)

TheModelClass(
  (l1): Linear(in_features=3, out_features=2, bias=True)
  (f): Tanh()
  (l2): Linear(in_features=2, out_features=1, bias=True)
)


In [175]:
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [176]:
for i in iter(model.parameters()):
    print(i)

Parameter containing:
tensor([[0.3175, 0.0744, 0.5717],
        [0.5333, 0.1111, 0.0145]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.5730, -0.4037], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.3636,  0.4321]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.1475], dtype=torch.float64, requires_grad=True)


In [177]:
train_dataset = torch.tensor(df_25_nn_norm.iloc[:-3, :].reset_index(drop = True).to_numpy())
train_dataset

tensor([[-0.5723, -0.5273, -0.3368, -0.3032],
        [-0.5926,  0.0876, -0.6047, -0.4885],
        [-0.2542, -0.2415, -0.6253, -0.7605],
        [-0.0647, -1.4950, -0.2819,  0.6274],
        [ 0.0165,  0.5239, -0.0896, -0.4308],
        [-0.0918,  0.3738, -0.0071, -0.6246],
        [-0.0173, -0.4712, -0.1170, -0.2076],
        [-0.0985, -0.4047, -0.0415, -0.2184],
        [-0.1527, -0.3405, -0.1239, -0.3329],
        [-0.0444,  0.2489, -0.1788, -0.6249],
        [-0.0782, -0.7292, -0.0690,  0.0629],
        [-0.3354,  0.1877, -0.1033, -0.7321],
        [-0.4302, -0.2416, -0.3643, -0.5855],
        [-0.1392,  0.4950, -0.4604, -0.2088],
        [ 0.2060,  1.7551, -0.1651,  0.7571],
        [ 0.1045,  1.1982,  0.1852,  0.0079],
        [ 0.0097,  0.3037,  0.0821, -0.7492],
        [ 0.1248,  0.8685, -0.0140, -0.1631],
        [ 0.2466, -0.1366,  0.1027, -0.3694],
        [ 4.6052,  3.1664,  0.2264,  1.7981],
        [ 0.5106, -1.4739,  4.6495,  3.9138],
        [-0.4640, -1.2774,  0.4942

In [178]:
test_dataset = torch.tensor(df_25_nn_norm.iloc[-3:, :].reset_index(drop = True).to_numpy())
test_dataset

tensor([[-0.7821, -0.9577, -0.4948, -0.0115],
        [-0.8092, -0.5740, -0.8176, -0.5824],
        [-0.8972, -0.3382, -0.8451, -0.7205]], dtype=torch.float64)

In [179]:
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
#                                            batch_size=len(train_dataset), 
#                                            shuffle=False)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
#                                           batch_size=len(test_dataset),
#                                           shuffle=False)

In [180]:
optimizer.param_groups[0]['lr']

0.01

In [181]:
with torch.no_grad():
    num_epochs = 1000000
    optimizer.param_groups[0]['lr'] = 0.01
    best_model = copy.deepcopy(model)
    best_loss_test = criterion(best_model(test_dataset[:,1:]), torch.unsqueeze(test_dataset[:,0], 1)).detach()
    i = 0
for epoch in range(num_epochs):
    outputs_train = model(train_dataset[:,1:])
    loss_train = criterion(outputs_train, torch.unsqueeze(train_dataset[:,0], 1))

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    
    with torch.no_grad():
        loss_test = criterion(model(test_dataset[:,1:]), torch.unsqueeze(test_dataset[:,0], 1))
        best_loss_test = criterion(best_model(test_dataset[:,1:]), torch.unsqueeze(test_dataset[:,0], 1))
    
        if epoch == 0:
            learning_rate = optimizer.param_groups[0]['lr']
            print (f'Epoch [{epoch+1}], Learning Rate: [{learning_rate:.4}], Train-Loss: [{loss_test:.4}], Test-Loss: [{loss_train:.4}]')
            
        if loss_test < best_loss_test - 1e-9:   
            best_model = copy.deepcopy(model)
            best_loss_test = loss_test
            i = 0
            learning_rate = optimizer.param_groups[0]['lr']
            print (f'Epoch [{epoch+1}], Learning Rate: [{learning_rate:.4}], Train-Loss: [{loss_train:.4}], Test-Loss: [{loss_test:.4}]')
        else:
            i += 1

        if i == 1000 and optimizer.param_groups[0]['lr'] / 2 > 1e-9:
            
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] / 2
            learning_rate = optimizer.param_groups[0]['lr']
            print (f'Epoch [{epoch+1}], Learning Rate: [{learning_rate:.4}], Train-Loss: [{loss_train:.4}]')
            i = 0
        elif optimizer.param_groups[0]['lr'] / 2 < 1e-9:
            learning_rate = optimizer.param_groups[0]['lr']
            print(f"На {epoch+1} эпохе Learning Rate стал слишком мал - {learning_rate:.4}") 
            break
            
        if num_epochs == 100000 - 1:
            print("Все эпохи прошли")            
        
for parameter in best_model.named_parameters():
    print(parameter)

Epoch [1], Learning Rate: [0.01], Train-Loss: [0.7092], Test-Loss: [0.4613]
Epoch [1], Learning Rate: [0.01], Train-Loss: [0.4613], Test-Loss: [0.7092]
Epoch [2], Learning Rate: [0.01], Train-Loss: [0.4583], Test-Loss: [0.6933]
Epoch [3], Learning Rate: [0.01], Train-Loss: [0.4524], Test-Loss: [0.6708]
Epoch [4], Learning Rate: [0.01], Train-Loss: [0.4451], Test-Loss: [0.6443]
Epoch [5], Learning Rate: [0.01], Train-Loss: [0.4364], Test-Loss: [0.6166]
Epoch [6], Learning Rate: [0.01], Train-Loss: [0.4289], Test-Loss: [0.5927]
Epoch [7], Learning Rate: [0.01], Train-Loss: [0.4224], Test-Loss: [0.5745]
Epoch [8], Learning Rate: [0.01], Train-Loss: [0.4148], Test-Loss: [0.5616]
Epoch [9], Learning Rate: [0.01], Train-Loss: [0.4062], Test-Loss: [0.551]
Epoch [10], Learning Rate: [0.01], Train-Loss: [0.3971], Test-Loss: [0.5424]
Epoch [11], Learning Rate: [0.01], Train-Loss: [0.3874], Test-Loss: [0.5356]
Epoch [12], Learning Rate: [0.01], Train-Loss: [0.3771], Test-Loss: [0.5303]
Epoch [13]

In [198]:
best_model(train_dataset[:,1:])

tensor([[-0.2852],
        [-0.1918],
        [-0.3066],
        [-0.4316],
        [ 0.0138],
        [-0.0297],
        [-0.2237],
        [-0.1948],
        [-0.2038],
        [-0.0911],
        [-0.2548],
        [-0.1045],
        [-0.2446],
        [-0.0351],
        [ 0.4340],
        [ 0.2771],
        [-0.0447],
        [ 0.1416],
        [-0.1164],
        [ 0.9656],
        [ 0.7431],
        [-0.2114]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [191]:
best_model(test_dataset[:,1:])

tensor([[-0.3936],
        [-0.4071],
        [-0.3650]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [209]:
df_25_nn_predict = best_model(torch.tensor(df_25_nn_norm.reset_index(drop = True).to_numpy())[:,1:])
df_25_nn_predict

tensor([[-0.2852],
        [-0.1918],
        [-0.3066],
        [-0.4316],
        [ 0.0138],
        [-0.0297],
        [-0.2237],
        [-0.1948],
        [-0.2038],
        [-0.0911],
        [-0.2548],
        [-0.1045],
        [-0.2446],
        [-0.0351],
        [ 0.4340],
        [ 0.2771],
        [-0.0447],
        [ 0.1416],
        [-0.1164],
        [ 0.9656],
        [ 0.7431],
        [-0.2114],
        [-0.3936],
        [-0.4071],
        [-0.3650]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [192]:
with torch.no_grad():
    print(criterion(best_model(test_dataset[:,1:]), torch.unsqueeze(test_dataset[:,0], 1)))
    print(best_model(test_dataset[:,1:]))
    print(torch.unsqueeze(test_dataset[:,0], 1))

tensor(0.4409, dtype=torch.float64)
tensor([[-0.3936],
        [-0.4071],
        [-0.3650]], dtype=torch.float64)
tensor([[-0.7821],
        [-0.8092],
        [-0.8972]], dtype=torch.float64)


In [193]:
for parameter in best_model.named_parameters():
    print(parameter[1][0].detach().numpy())

[ 0.28703257 -0.05268636  0.40314282]
-0.6211049695222318
[-0.11457248  0.51355737]
0.03135635463315049


In [194]:
torch.save(best_model, "NN_Tanh_3_1.pth")

In [195]:
best_model = torch.load('NN_Tanh_3_1.pth')

In [217]:
df_25_nn_norm.iloc[:, 1]

0    -0.527321
1     0.087555
2    -0.241532
3    -1.494985
4     0.523904
5     0.373840
6    -0.471233
7    -0.404712
8    -0.340545
9     0.248945
10   -0.729212
11    0.187672
12   -0.241563
13    0.495008
14    1.755115
15    1.198189
16    0.303723
17    0.868526
18   -0.136604
19    3.166397
20   -1.473878
21   -1.277444
22   -0.957699
23   -0.573953
24   -0.338192
Name: MoM, dtype: float64

In [220]:
df_25_nn_norm_predict = pd.DataFrame({'Russia_MoM_predict': pd.DataFrame(df_25_nn_predict.detach().numpy()).iloc[:, 0], 'MoM': df_25_nn_norm.iloc[:, 1], 'Russia_MoM-1': df_25_nn_norm.iloc[:, 2], 'Spread': df_25_nn_norm.iloc[:, 3]})
df_25_nn_norm_predict

,Russia_MoM_predict,MoM,Russia_MoM-1,Spread
0,-0.285185,-0.527321,-0.336815,-0.303152
1,-0.191828,0.087555,-0.604673,-0.488467
2,-0.306606,-0.241532,-0.625278,-0.760491
3,-0.431589,-1.494985,-0.281869,0.627438
4,0.013845,0.523904,-0.089561,-0.430762
5,-0.029691,0.373840,-0.007143,-0.624614
6,-0.223742,-0.471233,-0.117034,-0.207621
7,-0.194837,-0.404712,-0.041484,-0.218415
8,-0.203838,-0.340545,-0.123902,-0.332932
9,-0.091103,0.248945,-0.178847,-0.624865


In [223]:
df_25_nn_predict = pd.DataFrame(scaler_nn.inverse_transform(df_25_nn_norm_predict), columns=df_25_nn_norm_predict.columns)
df_25_nn_predict

,Russia_MoM_predict,MoM,Russia_MoM-1,Spread
0,100.384228,99.337024,100.35,1.012976
1,100.522166,100.570486,99.96,0.610486
2,100.352579,99.910327,99.93,0.019673
3,100.167912,97.395858,100.43,3.034142
4,100.826057,101.445817,100.71,0.735817
5,100.761730,101.144785,100.83,0.314785
6,100.475013,99.449539,100.67,1.220461
7,100.517722,99.582982,100.78,1.197018
8,100.504421,99.711704,100.66,0.948296
9,100.670992,100.894241,100.58,0.314241


In [227]:
mean_absolute_error(df_25.iloc[-3:, 0], df_25_nn_predict.iloc[-3:, 0])

0.6514721436781249